# Analyze Portfolio via a standard process

- Gather stocks of interest
- Gather stocks from specific criteria (SP500 top 50...)
- Gather stocks from specific portfolio account
- Assemble stock universe 
- Gather price histories
- Compute aplha factors (X)
    - Compute custom apha factors
    - Compute univeral quant factors
    - Compute date information
- Compute target values (y)
    - Quantize with 2 bins for testing
    - Quantize with 25 bins for production
- Generate Optimal Portfolio
- Report on new portfolio

In [15]:
from platform import python_version
import time
from datetime import datetime
import os
import pandas as pd
import numpy as np
import math
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (20, 8)

import importlib
import trading_functions as tf
importlib.reload(tf)
from trading_functions import Returns
from trading_functions import Data

import utils
importlib.reload(utils)

portfun = tf.Portfolio()
selfun = tf.Selection()

import portfolio_optimizer
from portfolio_optimizer import OptimalHoldings
importlib.reload(portfolio_optimizer)

import ameritrade_functions as amc
importlib.reload(amc)

print(f'Python version: {python_version()}')
print(f'Pandas version: {pd.__version__}')

Python version: 3.6.12
Pandas version: 0.25.3


## Configure Ameritrade Information

Ameritrade credentials are stored in environment variables to keep from having unencrypted passwords stored on disk.

The module automatically masks the account numbers to protect the actual accounts. An Ameritrade user can have many investment accounts. We will be working with only one for this demonstration.

In [6]:
username = os.getenv('maiotradeuser')
password = os.getenv('maiotradepw')
client_id = os.getenv('maiotradeclientid')

masked_account_number = '#---5311'
account_portfolios_file_name = 'data/portfolio_data.csv'
portfolio_file_name = 'data/portfolio_' + masked_account_number[-4:] + '.csv'
price_histories_file_name = 'data/price_histories.csv'

## Authentication Tokens

To get data from Ameritrade you will need to obtains a short time use token (there is a re-use token, but that has not been coded yet.)

To obtain a token, you will need to have a Chrome driver located somewhere on your system. This will allow the module to use your credentials to obtain an authentication token.

In [3]:
td_ameritrade = amc.AmeritradeRest(username, password, client_id)
td_ameritrade.authenticate()

if len(td_ameritrade.authorization) == 0:
    print('Error: No authorization data: {}'.format(td_ameritrade.authorization))
else:
    print('You have authorization')

You have authorization


## Stock Universe

Here we setup the univers. This needs some work. The long term goal is to use a pipeline process to help select stock that are in the top 500 or something similare.

For now we will use stocks from the portfolio, but stocks of interest (high news items), a list of well known stocks (this also has been augmented with some stocks that made Ameritrade's top 10 movers for a couple of days. This Ameritrade funciton has not been coded yet, but should be add down the line to automate pulling these tickers.

In [4]:
from trading_functions import Data

symbols_of_interest = ['MGM', 'PDYPF', 'NNXPF', 'WKHS']
# Hardcoded for now
symbols_via_specific_criteria = ['AAPL',
                                 'MSFT',
                                 'GOOG',
                                 'TSLA',
                                 'COKE',
                                 'IBM',
                                 'BABA',
                                 'GMGMF',
                                 'OEG',
                                 'LX',
                                 'AIH',
                                 'NMRD',
                                 'CAN',
                                 'MOSY',
                                 'QFIN',
                                 'OCG',
                                 'PRTK',
                                 'ZKIN', 
                                 'EFOI',
                                 'CONN',
                                 'LEDS',
                                 'TELL',
                                 'JZXN',
                                 'VTNR',
                                 'AEI',
                                 'IPQ',
                                 'RCON'
                                ]

# Specific Portfolio Account
account_portfolio_df = Data().get_account_portfolio_data(td_ameritrade.parse_portfolios_list(), masked_account_number)
equity_investments_df = portfun.get_investments_by_type(account_portfolio_df, investment_type='EQUITY')
symbols_from_account = list(equity_investments_df['symbol'].values)

stock_universe = set(symbols_of_interest + symbols_via_specific_criteria + symbols_from_account)

## Portfolio weights

With the portfolio stocks and the additional stocks, show how each of them contribute to the portfolio. Later, once we produce an optimized portfolio, we can generate a report on how much stock to buy/sell based of what we already have.

In [5]:
non_portfolio_symbols = stock_universe - set(symbols_from_account)
non_portfolio_values = pd.DataFrame.from_dict({ symbol : [0, 0] for symbol in non_portfolio_symbols}, orient='index')
non_portfolio_values.index.name='symbol'
non_portfolio_values.columns = ['marketValue', 'longQuantity']
stock_universe_values = portfun.get_market_values(equity_investments_df).append(non_portfolio_values)
sotck_universe_weights = portfun.get_portfolio_weights(stock_universe_values)
display(sotck_universe_weights)

symbol
AAPL     0.000000
AEI      0.000000
AIH      0.000000
BABA     0.000000
CAN      0.000000
COKE     0.000000
CONN     0.000000
DKNG     0.024709
EEENF    0.000040
EFOI     0.000000
FGPHF    0.080349
FNCL     0.033384
GMGMF    0.000000
GOOG     0.000000
IBM      0.000000
IPQ      0.000000
JZXN     0.000000
LEDS     0.000000
LX       0.000000
MGM      0.000000
MOSY     0.000000
MSFT     0.000000
NMRD     0.000000
NNXPF    0.000000
OCG      0.000000
OEG      0.000000
PDYPF    0.000000
PRTK     0.000000
QFIN     0.000000
RCON     0.000000
SCHA     0.020599
SCHF     0.087475
SCHM     0.060335
TELL     0.000000
TLRY     0.144831
TSLA     0.000000
VBK      0.018524
VBR      0.021686
VGK      0.091358
VTI      0.355585
VTNR     0.000000
VXF      0.061123
WKHS     0.000000
ZKIN     0.000000
dtype: float64

# Price History data

One you have a set of investments you want to work with, you will need to pull some historical data for them.

In [ ]:
number_of_years = 5
price_histories = td_ameritrade.get_price_histories(stock_universe, datetime.today().strftime('%Y-%m-%d'), num_periods=number_of_years)
utils.save_price_histories(price_histories, price_histories_file_name)

In [19]:
price_histories.head()

,open,high,low,close,volume,ticker,date
0,8.3000,8.75,7.0200,7.04,125031,WKHS,2016-06-13
1,8.1700,8.75,8.0600,8.08,652104,CONN,2016-06-13
2,107.1500,107.68,106.5900,106.61,2378771,VTI,2016-06-13
3,123.6800,124.47,122.7810,122.95,137315,VBK,2016-06-13
4,1.8933,1.98,1.8412,1.98,4325,LEDS,2016-06-13


In [16]:
price_histories = utils.read_price_histories(price_histories_file_name)
pricing = Data().get_close_values(price_histories)

## Factors

In [66]:
import trading_factors as alpha_factors
importlib.reload(alpha_factors)
import alphalens as al

In [48]:
all_factors = pd.concat(
[
    alpha_factors.FactorMomentum(price_histories, 252).demean().rank().zscore().for_al(),
    alpha_factors.FactorMeanReversion(price_histories, 5).demean().rank().zscore().smoothed().rank().zscore().for_al(),
    alpha_factors.OvernightSentiment(price_histories, 5).demean().rank().zscore().smoothed(10).rank().zscore().for_al()
], axis=1)
all_factors.sort_index(inplace=True)

## Universal Quant Features

- Volatility 
- Dollar Volume
- Market Dispersion
- Market Volatility

#TODO: SMA for market disp/vol

In [49]:
all_factors = pd.concat(
[
    all_factors,
    alpha_factors.AnnualizedVolatility(price_histories, 20).rank().zscore().for_al(),
    alpha_factors.AnnualizedVolatility(price_histories, 120).rank().zscore().for_al(),
    alpha_factors.AverageDollarVolume(price_histories, 20).rank().zscore().for_al(),
    alpha_factors.AverageDollarVolume(price_histories, 120).rank().zscore().for_al(),
    alpha_factors.MarketDispersion(price_histories, 20).for_al(),
    alpha_factors.MarketDispersion(price_histories, 120).for_al(),
    alpha_factors.MarketVolatility(price_histories, 20).for_al(),
    alpha_factors.MarketVolatility(price_histories, 120).for_al()
], axis=1)
all_factors.sort_index(inplace=True)

In [53]:
alpha_factors.FactorDateParts(all_factors)
all_factors.head()

momentum_252_day_logret  \
date                      ticker                            
2016-06-24 00:00:00+00:00 AAPL                        NaN   
                          AEI                         NaN   
                          AIH                         NaN   
                          BABA                        NaN   
                          CAN                         NaN   

                                  mean_reversion_5_day_logret_smoothed  \
date                      ticker                                         
2016-06-24 00:00:00+00:00 AAPL                                     NaN   
                          AEI                                      NaN   
                          AIH                                      NaN   
                          BABA                                     NaN   
                          CAN                                      NaN   

                                  overnight_sentiment_5_day_smoothed  \
date                      ticker                                       
2016-06-24 00:00:00+00:00 AAPL                             -0.887075   
                          AEI                              -1.370935   
                          AIH                              -1.532221   
                          BABA                              1.370935   
                          CAN                              -1.612864   

                                  annualzed_volatility_20_day  \
date                      ticker                                
2016-06-24 00:00:00+00:00 AAPL                            NaN   
                          AEI                             NaN   
                          AIH                             NaN   
                          BABA                            NaN   
                          CAN                             NaN   

                                  annualzed_volatility_120_day  \
date                      ticker                                 
2016-06-24 00:00:00+00:00 AAPL                             NaN   
                          AEI                              NaN   
                          AIH                              NaN   
                          BABA                             NaN   
                          CAN                              NaN   

                                  average_dollar_volume_20_day  \
date                      ticker                                 
2016-06-24 00:00:00+00:00 AAPL                             NaN   
                          AEI                              NaN   
                          AIH                              NaN   
                          BABA                             NaN   
                          CAN                              NaN   

                                  average_dollar_volume_120_day  \
date                      ticker                                  
2016-06-24 00:00:00+00:00 AAPL                              NaN   
                          AEI                               NaN   
                          AIH                               NaN   
                          BABA                              NaN   
                          CAN                               NaN   

                                  market_dispersion20_day  \
date                      ticker                            
2016-06-24 00:00:00+00:00 AAPL                        NaN   
                          AEI                         NaN   
                          AIH                         NaN   
                          BABA                        NaN   
                          CAN                         NaN   

                                  market_dispersion120_day  \
date                      ticker                             
2016-06-24 00:00:00+00:00 AAPL                         NaN   
                          AEI                          NaN   
                          AIH                         

## Compute the target values (y)

In [67]:
all_factors = pd.concat(
[
    all_factors,
    alpha_factors.FactorReturnQuantiles(price_histories, 2, 5).for_al('5Day2Quant'),
    alpha_factors.FactorReturnQuantiles(price_histories, 25, 5).for_al('5Day25Quant')
], axis=1).dropna()
all_factors.sort_index(inplace=True)
all_factors.head()

ValueError: ("Bin edges must be unique: array([-0.12348106, -0.02700449, -0.0159891 , -0.00754347, -0.00099926,\n        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,\n        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,\n        0.00092687,  0.00482168,  0.00559886,  0.00665175,  0.00823758,\n        0.01591506,  0.02042562,  0.03478408,  0.05155882,  0.10536026,\n        0.42526774]).\nYou can drop duplicate edges by setting the 'duplicates' kwarg", 'occurred at index 2016-06-20 00:00:00+00:00')

In [56]:
returns = alpha_factors.FactorReturns(price_histories, 5).factor_data
returns

ticker,AAPL,AEI,AIH,BABA,CAN,COKE,CONN,DKNG,EEENF,EFOI,...,TLRY,TSLA,VBK,VBR,VGK,VTI,VTNR,VXF,WKHS,ZKIN
date,,,,,,,,,,,,,,,,,,,,,
2016-06-20 00:00:00+00:00,-0.023281,0.000000,0.000000,0.035160,0.000000,0.136415,0.050674,0.000000,0.000000,0.090746,...,0.000000,0.008364,0.005435,0.004847,0.016335,-0.000845,0.052056,0.006779,-0.018639,0.000000
2016-06-21 00:00:00+00:00,-0.016032,0.000000,0.000000,0.010870,0.000000,0.103079,0.061444,0.000000,0.000000,0.075378,...,0.000000,0.021401,0.008306,0.011639,0.064374,0.007522,0.060169,0.007506,-0.024304,0.000000
2016-06-22 00:00:00+00:00,-0.016504,0.000000,0.000000,0.004328,0.000000,0.057977,0.008680,0.000000,0.000000,0.044913,...,0.000000,-0.101642,0.001465,0.008112,0.058442,0.006310,0.194156,0.001408,-0.002890,0.000000
2016-06-23 00:00:00+00:00,-0.014976,0.000000,0.000000,0.011054,0.000000,-0.002922,0.004969,0.000000,0.000000,0.026580,...,0.000000,-0.104021,0.019454,0.027466,0.084440,0.017093,0.007605,0.020715,-0.012941,0.000000
2016-06-24 00:00:00+00:00,-0.020453,0.000000,0.000000,-0.009264,0.000000,0.000427,-0.075954,0.000000,0.000000,0.030877,...,0.000000,-0.109355,-0.012444,-0.016210,-0.052338,-0.016775,-0.046520,-0.019261,0.033120,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-07 00:00:00+00:00,0.010299,0.124585,0.040868,0.013647,0.180335,0.101238,0.225297,0.068846,0.022728,0.220241,...,0.168023,-0.032660,0.006620,0.015495,0.014828,0.006763,0.129522,0.014392,0.397249,0.131825
2021-06-08 00:00:00+00:00,0.019601,0.180729,-0.327213,-0.016816,-0.083297,0.074678,0.207535,0.109387,0.028988,0.206017,...,0.174636,-0.033095,0.017881,0.013142,0.007028,0.008398,-0.011710,0.016069,0.481105,-0.035577
2021-06-09 00:00:00+00:00,0.016417,0.109519,-0.178448,-0.028969,-0.019163,0.060822,0.186409,0.066596,0.048202,0.145119,...,0.120353,-0.010533,0.016252,0.001732,0.003873,0.004248,-0.033902,0.010600,0.383229,0.140331


In [58]:
pd.DataFrame(pd.qcut(returns, 2, labels=False, duplicates='drop'), index=returns.index, columns=returns.columns)

ValueError: Input array must be 1 dimensional

In [65]:
returns.apply(lambda x: pd.qcut(x, 2, labels=False), axis=1)

ticker,AAPL,AEI,AIH,BABA,CAN,COKE,CONN,DKNG,EEENF,EFOI,...,TLRY,TSLA,VBK,VBR,VGK,VTI,VTNR,VXF,WKHS,ZKIN
date,,,,,,,,,,,,,,,,,,,,,
2016-06-20 00:00:00+00:00,0,0,0,1,0,1,1,0,0,1,...,0,1,1,1,1,0,1,1,0,0
2016-06-21 00:00:00+00:00,0,0,0,1,0,1,1,0,0,1,...,0,1,1,1,1,1,1,1,0,0
2016-06-22 00:00:00+00:00,0,0,0,1,0,1,1,0,0,1,...,0,0,1,1,1,1,1,1,0,0
2016-06-23 00:00:00+00:00,0,0,0,1,0,0,1,0,0,1,...,0,0,1,1,1,1,1,1,0,0
2016-06-24 00:00:00+00:00,0,1,1,0,1,1,0,1,1,1,...,1,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-07 00:00:00+00:00,0,1,0,0,1,1,1,1,0,1,...,1,0,0,0,0,0,1,0,1,1
2021-06-08 00:00:00+00:00,1,1,0,0,0,1,1,1,1,1,...,1,0,0,0,0,0,0,0,1,0
2021-06-09 00:00:00+00:00,0,1,0,0,0,1,1,1,1,1,...,1,0,0,0,0,0,0,0,1,1


In [61]:
returns.quantile(.5, axis='index')

ticker
AAPL     0.009449
AEI      0.000000
AIH      0.000000
BABA     0.007504
CAN      0.000000
COKE     0.002569
CONN     0.000000
DKNG     0.000000
EEENF    0.000000
EFOI    -0.012686
FGPHF    0.000000
FNCL     0.003967
GMGMF    0.000000
GOOG     0.007323
IBM      0.001378
JZXN     0.000000
LEDS    -0.003155
LX       0.000000
MGM      0.004142
MOSY    -0.012274
MSFT     0.007080
NMRD    -0.003096
NNXPF    0.000000
OCG      0.000000
OEG      0.000000
PDYPF    0.000000
PRTK    -0.004255
QFIN     0.000000
RCON    -0.007489
SCHA     0.004520
SCHF     0.003234
SCHM     0.004126
TELL    -0.003529
TLRY     0.000000
TSLA     0.010026
VBK      0.005717
VBR      0.003088
VGK      0.003133
VTI      0.004612
VTNR    -0.006645
VXF      0.005249
WKHS    -0.003630
ZKIN     0.000000
Name: 0.5, dtype: float64

In [ ]:
target_source = '5Day2Quant'
all_factors['target'] = all_factors.groupby(level='ticker')[target_source].shift(-5)
all_factors[[target_source,'target']].reset_index().sort_values(['ticker', 'date']).head(10)

### IID Check of Target
Let's see if the returns are independent and identically distributed.

In [ ]:
from scipy.stats import spearmanr

def sp(group, col1_name, col2_name):
    x = group[col1_name]
    y = group[col2_name]
    return spearmanr(x, y)[0]


all_factors['target_1'] = all_factors.groupby(level='ticker')[target_source].shift(-4)
all_factors['target_2'] = all_factors.groupby(level='ticker')[target_source].shift(-3)
all_factors['target_3'] = all_factors.groupby(level='ticker')[target_source].shift(-2)
all_factors['target_4'] = all_factors.groupby(level='ticker')[target_source].shift(-1)

g = all_factors.dropna().groupby(level='date')
for i in range(4):
    label = 'target_'+str(i+1)
    ic = g.apply(sp, 'target', label)
    ic.plot(ylim=(-1, 1), label=label)
plt.legend(bbox_to_anchor=(1.04, 1), borderaxespad=0)
plt.title('Rolling Autocorrelation of Labels Shifted 1,2,3,4 Days')
plt.show()

In [ ]:
all_factors.to_csv('data/factor_features_data.csv')

In [ ]:
all_factors = pd.read_csv('data/factor_features_data.csv')
all_factors.set_index(['date', 'ticker'], inplace=True)
all_factors.sort_index(inplace=True)
all_factors.head()

In [ ]:
all_factors.columns.values

In [ ]:
import utils
importlib.reload(utils)

features = ['momentum_252_day_logret', 'mean_reversion_5_day_logret_smoothed',
       'overnight_sentiment_5_day_smoothed',
       'annualzed_volatility_20_day', 'annualzed_volatility_120_day',
       'average_dollar_volume_20_day', 'average_dollar_volume_120_day',
       'market_dispersion20_day', 'market_dispersion120_day',
       'market_volatility20_day', 'market_volatility120_day',
       'is_January', 'is_December', 'weekday', 'quarter', 'year',
       'month_start', 'month_end', 'quarter_start', 'quarter_end']

target_label = 'target'

temp = all_factors.dropna().copy()
X = temp[features]
y = temp[target_label]

X_train, X_valid, X_test, y_train, y_valid, y_test = utils.train_valid_test_split(X, y, 0.6, 0.2, 0.2)

X_train.head()

In [ ]:
from IPython.display import display
from sklearn.tree import DecisionTreeClassifier
import utils
importlib.reload(utils)


# This is to get consistent results between each run.
clf_random_state = 0

simple_clf = DecisionTreeClassifier(
    max_depth=3,
    criterion='entropy',
    random_state=clf_random_state)
simple_clf.fit(X_train, y_train)

display(utils.plot_tree_classifier(simple_clf, feature_names=features))
utils.rank_features_by_importance(simple_clf.feature_importances_, features)

### Train Random Forests with Different Tree Sizes
Let's build models using different tree sizes to find the model that best generalizes.
#### Parameters
When building the models, we'll use the following parameters.

In [ ]:
n_days = 10
n_stocks = len(set(all_factors.index.get_level_values(level='ticker').values))

clf_parameters = {
    'criterion': 'entropy',
    'min_samples_leaf': n_stocks * n_days,
    'oob_score': True,
    'n_jobs': -1,
    'random_state': clf_random_state}
n_trees_l = [50, 100, 250, 500, 1000]

In [ ]:
from sklearn.ensemble import RandomForestClassifier

train_score = []
valid_score = []
oob_score = []
feature_importances = []

for n_trees in tqdm(n_trees_l, desc='Training Models', unit='Model'):
    clf = RandomForestClassifier(n_trees, **clf_parameters)
    clf.fit(X_train, y_train)
    
    train_score.append(clf.score(X_train, y_train.values))
    valid_score.append(clf.score(X_valid, y_valid.values))
    oob_score.append(clf.oob_score_)
    feature_importances.append(clf.feature_importances_)

In [ ]:
utils.plot_results(n_trees_l, train_score, oob_score, valid_score, ['train', 'oob', 'validation'], 'Random Forest Accuracy', 'Number of Trees')

In [ ]:
print('Features Ranked by Average Importance:\n')
ranked_features = utils.rank_features_by_importance(np.average(feature_importances, axis=0), features)

In [ ]:
ranked_features

In [ ]:
import utils
importlib.reload(utils)

train_score = []
valid_score = []
oob_score = []

for n_trees in tqdm(n_trees_l, desc='Training Models', unit='Model'):
    clf = RandomForestClassifier(n_trees, **clf_parameters)
    clf.fit(*utils.non_overlapping_samples(X_train, y_train, 4))
    
    train_score.append(clf.score(X_train, y_train.values))
    valid_score.append(clf.score(X_valid, y_valid.values))
    oob_score.append(clf.oob_score_)
    
utils.plot_results(n_trees_l, train_score, oob_score, valid_score, ['train', 'oob', 'validation'], 'Random Forest Accuracy - No Overlap', 'Number of Trees')

In [ ]:
import abc

from sklearn.ensemble import VotingClassifier
from sklearn.base import clone
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import Bunch


class NoOverlapVoterAbstract(VotingClassifier):
    @abc.abstractmethod
    def _calculate_oob_score(self, classifiers):
        raise NotImplementedError
        
    @abc.abstractmethod
    def _non_overlapping_estimators(self, x, y, classifiers, n_skip_samples):
        raise NotImplementedError
    
    def __init__(self, estimator, voting='soft', n_skip_samples=4):
        # List of estimators for all the subsets of data
        estimators = [('clf'+str(i), estimator) for i in range(n_skip_samples + 1)]
        
        self.n_skip_samples = n_skip_samples
        super().__init__(estimators, voting)
    
    def fit(self, X, y, sample_weight=None):
        estimator_names, clfs = zip(*self.estimators)
        self.le_ = LabelEncoder().fit(y)
        self.classes_ = self.le_.classes_
        
        clone_clfs = [clone(clf) for clf in clfs]
        self.estimators_ = self._non_overlapping_estimators(X, y, clone_clfs, self.n_skip_samples)
        self.named_estimators_ = Bunch(**dict(zip(estimator_names, self.estimators_)))
        self.oob_score_ = self._calculate_oob_score(self.estimators_)
        
        return self

You might notice that two of the functions are abstracted. These will be the functions that you need to implement.
#### OOB Score
In order to get the correct OOB score, we need to take the average of all the estimator's OOB scores. Implement `calculate_oob_score` to calculate this score.

In [ ]:
def calculate_oob_score(classifiers):
    """
    Calculate the mean out-of-bag score from the classifiers.

    Parameters
    ----------
    classifiers : list of Scikit-Learn Classifiers
        The classifiers used to calculate the mean out-of-bag score
    
    Returns
    -------
    oob_score : float
        The mean out-of-bag score
    """
    
    # TODO: Implement
    oob_scores = []
    for classifier in classifiers:
        oob_scores.append(classifier.oob_score_)
    
    return np.mean(oob_scores)

#### Non Overlapping Estimators
With `calculate_oob_score` implemented, let's create non overlapping estimators. Implement `non_overlapping_estimators` to build non overlapping subsets of the data, then run a estimator on each subset of data.

In [ ]:
def non_overlapping_estimators(x, y, classifiers, n_skip_samples):
    """
    Fit the classifiers to non overlapping data.

    Parameters
    ----------
    x : DataFrame
        The input samples
    y : Pandas Series
        The target values
    classifiers : list of Scikit-Learn Classifiers
        The classifiers used to fit on the non overlapping data
    n_skip_samples : int
        The number of samples to skip
    
    Returns
    -------
    fit_classifiers : list of Scikit-Learn Classifiers
        The classifiers fit to the the non overlapping data
    """
    
    # TODO: Implement

    fit_classifiers = []
    for clf_idx in range(len(classifiers)):
        x_samp, y_samp = utils.non_overlapping_samples(x, y, n_skip_samples, clf_idx)
        fit_classifiers.append(classifiers[clf_idx].fit(x_samp, y_samp))
    
    return fit_classifiers

In [ ]:
class NoOverlapVoter(NoOverlapVoterAbstract):
    def _calculate_oob_score(self, classifiers):
        return calculate_oob_score(classifiers)
        
    def _non_overlapping_estimators(self, x, y, classifiers, n_skip_samples):
        return non_overlapping_estimators(x, y, classifiers, n_skip_samples)

Now that we have our `NoOverlapVoter` class, let's train it.

#### Train Model

In [ ]:
train_score = []
valid_score = []
oob_score = []

for n_trees in tqdm(n_trees_l, desc='Training Models', unit='Model'):
    clf = RandomForestClassifier(n_trees, **clf_parameters)
    
    clf_nov = NoOverlapVoter(clf, voting='soft')
    clf_nov.fit(X_train, y_train)
    
    train_score.append(clf_nov.score(X_train, y_train.values))
    valid_score.append(clf_nov.score(X_valid, y_valid.values))
    oob_score.append(clf_nov.oob_score_)
    
utils.plot_results(n_trees_l, train_score, oob_score, valid_score, ['train', 'oob', 'validation'], 'No Overlapping Voter', 'Number of Trees')

In [ ]:
import alphalens as al
importlib.reload(al)
import utils
importlib.reload(utils)
import trading_factors as alpha_factors
importlib.reload(alpha_factors)
print(al.__version__)

all_assets = all_factors.index.levels[1].values.tolist()
all_pricing = price_histories

def show_sample_results(data, samples, classifier, factors, pricing=all_pricing):
    # Calculate the Alpha Score
    prob_array=[-1,1]
    alpha_score = classifier.predict_proba(samples).dot(np.array(prob_array))
    
    # Add Alpha Score to rest of the factors
    alpha_score_label = 'AI_ALPHA'
    factors_with_alpha = data.loc[samples.index].copy().reset_index()
    factors_with_alpha['date'] = pd.to_datetime(factors_with_alpha['date'])
    factors_with_alpha.set_index(['date', 'ticker'], inplace=True)
    factors_with_alpha = factors_with_alpha.tz_localize('UTC', level='date')
    factors_with_alpha[alpha_score_label] = alpha_score
    
    # Setup data for AlphaLens
    print('Cleaning Data...\n')
    factor_data = alpha_factors.build_factor_data(factors_with_alpha[factors + [alpha_score_label]], pricing)
    print('\n-----------------------\n')
    
    # Calculate Factor Returns and Sharpe Ratio
    factor_returns = alpha_factors.get_factor_returns(factor_data)
    sharpe_ratio = alpha_factors.sharpe_ratio(factor_returns)
    
    # Show Results
    print('             Sharpe Ratios')
    print(sharpe_ratio.round(2))
    project_helper.plot_factor_returns(factor_returns)
    project_helper.plot_factor_rank_autocorrelation(factor_data)

In [ ]:
features_to_use = 0.25
features_count = int(features_to_use * len(ranked_features))
factor_names = ranked_features[:features_count]
print(factor_names)
show_sample_results(all_factors.dropna(), X_train.dropna(), clf_nov, factor_names, Data().get_close_values(price_histories))

Let's look at the accuracy of the classifiers over the number of trees.

In [ ]:
import shap
shap.initjs() #initialize javascript to enable visualizations

def model_shap_importances(model, features, X, tree_limit=50):
    """
    Note that the observations should be numeric (integer or float).
    So booleans should be converted to 1 (True) and 0 (False) 
    """
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X, tree_limit=tree_limit, check_additivity=False)
    
    shap_values_matrix = np.concatenate(shap_values)
    
    shap_abs = np.abs(shap_values_matrix)
    
    global_importances = np.nanmean(shap_abs, axis=0)
        
    indices = np.argsort(global_importances)[::-1]
    features_ranked = []
    for f in range(X.shape[1]):
        print("%d. %s (%d) (%f)" % (f+1,features[indices[f]], indices[f], global_importances[indices[f]]))
        features_ranked.append(features[indices[f]])
        
    return features_ranked

In [ ]:
features_ranked = model_shap_importances(clf,features,X_train,10)

In [ ]:
explainer = shap.TreeExplainer(clf)
shap_values = explainer.shap_values(X_train, tree_limit=n_trees_l[-1])
shap.summary_plot(shap_values, X_train, plot_type="bar")

# Stopped here

### View Data
With the `OptimalHoldings` class implemented, let's see the weights it generates.

In [ ]:
optimal_weights = OptimalHoldings(risk_cap=0.05,weights_max=0.12, weights_min=-0.12).find(alpha_vector, risk_model.factor_betas_, risk_model.factor_cov_matrix_, risk_model.idiosyncratic_var_vector_)
print(f'Old portfolio variance is:  {risk_model.compute_portfolio_variance(sotck_universe_weights):.8f}')
print(f'New portfolio variance is:  {risk_model.compute_portfolio_variance(optimal_weights):.8f}')
display((optimal_weights*100).round(2))
optimal_weights.plot.bar(legend=None, title='Portfolio % Holdings by Stock')

#x_axis = plt.axes().get_xaxis()
#x_axis.set_visible(False)

In [ ]:
investment_amount = portfun.get_account_value(stock_universe_values).round(2)
print(f'Current investment amount: {investment_amount}')
optimal_weights['amount'] = (optimal_weights['optimalWeights'] * investment_amount).round(0)
optimal_weights['marketValue'] = stock_universe_values['marketValue']
optimal_weights['buy/sell'] = (optimal_weights['marketValue'] - optimal_weights['amount']) * -1
optimal_weights['close'] = pricing.iloc[-1]
optimal_weights['existingShares'] = stock_universe_values['longQuantity']
optimal_weights['deltaShares'] = (optimal_weights['buy/sell'] / optimal_weights['close']).round(0)
optimal_weights['deltaMarketValue'] = (optimal_weights['deltaShares'] * optimal_weights['close'])
optimal_weights['totalShares'] = (optimal_weights['existingShares'] + optimal_weights['deltaShares'])
optimal_weights['totalMarketValue'] = (optimal_weights['totalShares'] * optimal_weights['close'])
optimal_weights

## Think or Swim Simulator

In [ ]:
optimal_weights[(optimal_weights['totalShares'] > 0) | (optimal_weights['deltaShares'] != 0)].round(2)

In [ ]:
optimal_weights[(optimal_weights['totalShares'] <= 0) & (optimal_weights['deltaShares'] == 0)].round(2)

In [ ]:
optimal_weights[(optimal_weights['totalShares'] > 0) | (optimal_weights['deltaShares'] != 0)][['deltaMarketValue', 'totalMarketValue']].sum().round(2)